In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [4]:
from typing import List, Type
from pydantic import BaseModel, create_model

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(
                    model='gemini-1.5-flash',
                    max_output_tokens=2048,
                    temperature=0.7,
                    top_p=0.9
                )

C:\Users\rahul\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
corpse = "What is Generative AI"

In [7]:
prompt = [
    ("system", f"Extract information from the following text and return ONLY a JSON object"),
    ("human", f"Answer this question - {corpse}")
]

In [8]:
response=model.invoke("What is Data Science?")

In [10]:
response2 = model.invoke(prompt)

KeyboardInterrupt: 

In [11]:
from langchain_core.output_parsers import StrOutputParser
result = StrOutputParser().invoke(response)

In [ ]:
result2 = StrOutputParser().invoke(response2)

In [12]:
result

"Data science is an interdisciplinary field that uses scientific methods, processes, algorithms and systems to extract knowledge and insights from structured and unstructured data.  It combines aspects of:\n\n* **Mathematics and Statistics:**  Used for modeling, analysis, and drawing inferences from data.  This includes probability, linear algebra, calculus, and statistical inference.\n\n* **Computer Science:**  Needed for data storage, retrieval, cleaning, processing, and building algorithms and systems to handle large datasets.  This involves programming skills (Python, R, SQL are common), database management, and distributed computing.\n\n* **Domain Expertise:**  Understanding the context of the data is crucial for interpreting results and asking the right questions.  This varies widely depending on the application (e.g., healthcare, finance, marketing).\n\nIn essence, data scientists use their skills to:\n\n* **Collect data:** From various sources, both internal and external to an 

In [ ]:
result2

'```json\n{\n  "Generative AI": "Generative AI is a type of artificial intelligence that can create new content, such as text, images, audio, and video.  It learns patterns from input data and then uses that knowledge to generate similar but novel outputs."\n}\n```\n'

In [13]:
def create_dynamic_listing_model(field_names: List[str]) -> Type[BaseModel]:
    """
    Dynamically creates a Pydantic model based on provided fields.
    field_name is a list of names of the fields to extract from the markdown.
    """
    # Create field definitions using aliases for Field parameters
    field_definitions = {field: (str, ...) for field in field_names}
    # Dynamically create the model with all field
    return create_model('DynamicListingModel', **field_definitions)


def create_listings_container_model(listing_model: Type[BaseModel]) -> Type[BaseModel]:
    """
    Create a container model that holds a list of the given listing model.
    """
    return create_model('DynamicListingsContainer', listings=(List[listing_model], ...))

In [14]:
fields = ['City']

In [15]:
DynamicListingModel = create_dynamic_listing_model(fields)
DynamicListingsContainer = create_listings_container_model(DynamicListingModel)

In [34]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader2=WebBaseLoader(web_path=("https://publiclibraries.com/state/alabama/"),
                      bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                          class_=("post-631 page type-page status-publish entry")
                      ))
                      )

In [36]:
data = loader2.load()

In [38]:
model2 = ChatGoogleGenerativeAI(
                    model='gemini-1.5-flash',
                    max_output_tokens=2048,
                    temperature=0.7,
                    top_p=0.9,
                    generation_config={
                        "response_mime_type": "application/json",
                        "response_schema": DynamicListingsContainer
                    }
                )

In [37]:
SYSTEM_MESSAGE = """You are an intelligent text extraction and conversion assistant. Your task is to extract structured information 
                        from the given text and convert it into a pure JSON format. The JSON should contain only the structured data extracted from the text, 
                        with no additional commentary, explanations, or extraneous information. 
                        You could encounter cases where you can't find the data of the fields you have to extract or the data will be in a foreign language.
                        Please process the following text and provide the output in pure JSON format with no words before or after the JSON:"""

USER_MESSAGE = f"Extract the following information from the provided text:\nPage content:\n\n"

In [40]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_MESSAGE,
        ),
        ("human", USER_MESSAGE),
    ]
)
chain = prompt | model2
chain.invoke("input": data)

SyntaxError: invalid syntax (2904960990.py, line 13)